### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!


- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [4]:
# !pip install langchain

In [114]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
# groq_api_key

In [116]:
# print(os.environ['OPENAI_API_KEY'])

In [21]:
# !pip install langchain_groq

In [23]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

In [25]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x113106c50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x113110f70>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [29]:
# !pip install langchain_core

In [43]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to hindi"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [44]:
result

AIMessage(content='नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n\n\nLet me break it down:\n\n* **नमस्ते (Namaste)** - Hello\n* **आप (aap)** - You (formal, respectful)\n* **कैसे (kaise)** - How\n* **हैं (hain)** - are \n\n\nThis translates to "Hello, how are you?" in a polite and respectful way.\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 21, 'total_tokens': 117, 'completion_time': 0.174545455, 'prompt_time': 0.002186936, 'queue_time': 0.243690716, 'total_time': 0.176732391}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--1d52e224-8bfe-4f11-9bf2-58fbbe52d83c-0', usage_metadata={'input_tokens': 21, 'output_tokens': 96, 'total_tokens': 117})

In [47]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n\n\nLet me break it down:\n\n* **नमस्ते (Namaste)** - Hello\n* **आप (aap)** - You (formal, respectful)\n* **कैसे (kaise)** - How\n* **हैं (hain)** - are \n\n\nThis translates to "Hello, how are you?" in a polite and respectful way.\n'

In [49]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n\n\n* **नमस्ते (Namaste)** means hello in Hindi.\n* **आप (aap)** means you (formal)\n* **कैसे (kaise)** means how\n* **हैं (hain)** means are \n\n'

In [65]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [67]:
result=prompt.invoke({"language":"Hindi","text":"my name is tejan"})

In [69]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into Hindi:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='my name is tejan', additional_kwargs={}, response_metadata={})]

In [89]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"thai","text":"my wife name is anita"})

'ชื่อภรรยาของฉันคือ อานิตา \n\n(Cheew phra-chana khon chan kee A-nita) \n\n\nLet me know if you need any other translations! 😊\n'

In [100]:
# !pip install langserve

In [102]:
# import os
# desktop_path = os.path.expanduser("/Users/tejan/Desktop/Dep Learning/LCEL/")
# print(desktop_path)

/Users/tejan/Desktop/Dep Learning/LCEL/


In [108]:
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
import os
from langserve import add_routes
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

parser=StrOutputParser()

##create chain
chain=prompt_template|model|parser



## App definition
app=FastAPI(title="Langchain Server",
            version="1.0",
            description="A simple API server using Langchain runnable interfaces")

## Adding chain routes
add_routes(
    app,
    chain,
    path="/chain"
)

if __name__=="__main__":
    import uvicorn
    uvicorn.run(app,host="localhost",port=8000)




RuntimeError: asyncio.run() cannot be called from a running event loop

In [110]:
!pip install nest_asyncio

In [112]:
import nest_asyncio
import uvicorn
nest_asyncio.apply()

uvicorn.run(app, host="localhost", port=8000)

INFO:     Started server process [7151]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 48] error while attempting to bind on address ('127.0.0.1', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.



     __          ___      .__   __.   _______      _______. _______ .______     ____    ____  _______
    |  |        /   \     |  \ |  |  /  _____|    /       ||   ____||   _  \    \   \  /   / |   ____|
    |  |       /  ^  \    |   \|  | |  |  __     |   (----`|  |__   |  |_)  |    \   \/   /  |  |__
    |  |      /  /_\  \   |  . `  | |  | |_ |     \   \    |   __|  |      /      \      /   |   __|
    |  `----./  _____  \  |  |\   | |  |__| | .----)   |   |  |____ |  |\  \----.  \    /    |  |____
    |_______/__/     \__\ |__| \__|  \______| |_______/    |_______|| _| `._____|   \__/     |_______|
    
LANGSERVE: Playground for chain "/chain/" is live at:
LANGSERVE:  │
LANGSERVE:  └──> /chain/playground/
LANGSERVE:
LANGSERVE: See all available routes at /docs/


SystemExit: 1

/Users/tejan/Desktop/GEN_AI/LANGCHAIN/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
